In [1]:
import os
import sys

#For safe imports of everything
notebook_directory = os.getcwd()
parent_directory = os.path.dirname(notebook_directory)
sys.path.insert(False, parent_directory)


In [2]:
from fam.llm.fast_inference import TTS

import warnings

# Filter out user warnings
warnings.filterwarnings("ignore")

/root/kotoba-speech-release/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using dtype=bfloat16


/root/kotoba-speech-release/.venv/lib/python3.10/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [3]:
model = TTS(model_name='metavoiceio/metavoice-1B-v0.1')

using dtype=bfloat16


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 37117.73it/s]


number of parameters: 14.07M
2024-04-26 07:59:48 | INFO     | DF | Running on torch 2.1.0+cu121
2024-04-26 07:59:48 | INFO     | DF | Running on host futuristic-hand-glows-fin-01
2024-04-26 07:59:48 | INFO     | DF | Git commit: 65a024e, branch: release
2024-04-26 07:59:48 | INFO     | DF | Loading model settings of DeepFilterNet3
2024-04-26 07:59:48 | INFO     | DF | Using DeepFilterNet3 model at /root/.cache/DeepFilterNet/DeepFilterNet3
2024-04-26 07:59:48 | INFO     | DF | Initializing model `deepfilternet3`
2024-04-26 07:59:48 | INFO     | DF | Found checkpoint /root/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2024-04-26 07:59:48 | INFO     | DF | Running on device cuda:0
2024-04-26 07:59:48 | INFO     | DF | Model loaded
/root/.cache/huggingface/hub/models--metavoiceio--metavoice-1B-v0.1/snapshots/d7237c5cb970da187623738f2957b338f3b0889a/first_stage.pt
Using device=cuda
Loading model ...
using dtype=bfloat16
Time to load model: 0.91 seconds


100%|██████████| 199/199 [00:03<00:00, 55.62it/s]

Compilation time: 4.03 seconds


## **Testing the new synthesizer** ##

In [4]:
model.synthesiseSequential(text="The university of waterloo is a publicly funded research university located in ontario canada.", spk_ref_path="/root/kotoba-speech-release/fam/llm/waterloov2.mp3")

 28%|██▊       | 320/1133 [00:05<00:14, 56.73it/s]


Time for 1st stage LLM inference: 5.70 sec total, 207.33 tokens/sec
Bandwidth achieved: 517.68 GB/s
Memory used: 13.44 GB

Truncating to min length of 590 tokens from 591 max.
[591, 590]
is_causal
in_x  torch.Size([1, 2, 1024])


non-causal batching: 100%|██████████| 1/1 [00:00<00:00, 33.51it/s]


idx torch.Size([1, 2, 1024])
audio input 6
431
probs 6
probs torch.Size([1, 1024, 1025])

Truncating to min length of 590 tokens from 643 max.
[590, 590, 643, 643, 643, 643, 643, 643]
Text: The university of waterloo is a publicly funded research university located in ontario canada.


2024-04-26 08:00:11 | WARNING  | DF | Audio sampling rate does not match model sampling rate (24000, 48000). Resampling...



Saved audio to /root/kotoba-speech-release/fam/outputs/synth_The_university_of_waterlo_8ee32452-d618-42c1-8162-363b2c70fb41.wav

Saved audio to /root/kotoba-speech-release/fam/outputs/synth_The_university_of_waterlo_8ee32452-d618-42c1-8162-363b2c70fb41.wav

Total time to synth (s): 8.46227240562439
Real-time factor: 1.08


'/root/kotoba-speech-release/fam/outputs/synth_The_university_of_waterlo_8ee32452-d618-42c1-8162-363b2c70fb41.wav'

In [5]:
model.synthesiseSequential(text="I have a black cat and waterloo is cool.", spk_ref_path="/root/kotoba-speech-release/fam/llm/black_cat.mp3")

 21%|██        | 368/1782 [00:06<00:24, 56.64it/s]


Time for 1st stage LLM inference: 6.53 sec total, 94.08 tokens/sec
Bandwidth achieved: 234.92 GB/s
Memory used: 13.79 GB

Truncating to min length of 306 tokens from 307 max.
[307, 306]
is_causal
in_x  torch.Size([1, 2, 1024])


non-causal batching: 100%|██████████| 1/1 [00:00<00:00, 139.78it/s]


idx torch.Size([1, 2, 1024])
audio input 6
123
probs 6
probs torch.Size([1, 1024, 1025])

Truncating to min length of 306 tokens from 326 max.
[306, 306, 326, 326, 326, 326, 326, 326]
Text: I have a black cat and waterloo is cool.

Saved audio to /root/kotoba-speech-release/fam/outputs/synth_I_have_a_black_cat_and_wa_c762b40f-a1a7-4f57-b7a2-28110c4f2f6e.wav

Saved audio to /root/kotoba-speech-release/fam/outputs/synth_I_have_a_black_cat_and_wa_c762b40f-a1a7-4f57-b7a2-28110c4f2f6e.wav

Total time to synth (s): 8.233639001846313
Real-time factor: 2.02


'/root/kotoba-speech-release/fam/outputs/synth_I_have_a_black_cat_and_wa_c762b40f-a1a7-4f57-b7a2-28110c4f2f6e.wav'

**Testing audio tokenizer**

In [6]:
from fam.llm.decoders import EncodecDecoder
from fam.llm.inference import TiltedEncodec
import torch
from pathlib import Path
from huggingface_hub import snapshot_download
from fam.quantiser.text.tokenise import TrainedBPETokeniser

data_adapter_second_stage = TiltedEncodec(end_of_audio_token=1024)
_other_model_dir = snapshot_download(repo_id="metavoiceio/metavoice-1B-v0.1")
checkpoint_path = Path(f"{_other_model_dir}/first_stage.pt")

checkpoint = torch.load(str(checkpoint_path), mmap=True, weights_only=False)
tokenizer_info = checkpoint.get("meta", {}).get("tokenizer", {})
tokenizer = TrainedBPETokeniser(**tokenizer_info)

enc = EncodecDecoder(data_adapter_fn=data_adapter_second_stage.decode, output_dir="/root/kotoba-speech-release/fam/outputs", tokeniser_decode_fn=tokenizer)

toks = enc.get_tokens(audio_path="/root/kotoba-speech-release/fam/llm/waterloostuffs.mp3")

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 25731.93it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/root/kotoba-speech-release/fam/llm/waterloostuffs.mp3'

In [ ]:
print(tokenizer.encode("Hey there goose man"))

[2453, 2170, 2517, 2347, 2404, 2365, 2482, 2561]


In [ ]:
len(toks)

8

In [ ]:
len(toks[0])

548